In [ ]:
import random
from PIL import Image
from google.colab.patches import cv2_imshow
import os
from google.colab import drive
import glob
drive.mount('/content/drive/')
drive = '/content/drive/MyDrive/Adrian/Semantic Segmentation-Laura Ulrich/Voronoi'

#Maskenbausteine1_path=os.path.join(drive, 'Maskenbausteine', 'Klasse1')
#Maskenbausteine2_path=os.path.join(drive, 'Maskenbausteine', 'Klasse2')

def get_files(directory, extension='*.jpg'):
    files = sorted(glob.glob(os.path.join(directory, extension)))
    print (files)
    return files

duktil_path = os.path.join(drive,'Bruchflächen','duktil (von Random Forest UP)')
duktil_path = get_files(duktil_path, '*.jpg')
spröde_path = os.path.join(drive,'Bruchflächen','spröde (von Random Forest UP)')
spröde_path = get_files(spröde_path, '*.jpg')

# Generated with SMOP  0.41

#function

def makelinefun(x,y, size):
  import numpy as np
  from numpy import linspace
  for k in range(0,len(x)):
    fun= np.linspace(x[k-1],x[k],size)
    fun2= np.linspace(y[k-1],y[k],size)
    for i in range(0,50):
      fun[i]=np.around(fun[i])
      fun2[i]=np.around(fun2[i])
    return fun, fun2

Mounted at /content/drive/
[]
[]


In [ ]:
#Generated with SMOP  0.41

#function
import matplotlib.pyplot as plt
def voronoi2mask(points=None,xpoints=None,ypoints=None,size=None,border=None,*args,**kwargs):


# voronoi2mask.m:81
    from scipy.spatial import Voronoi, voronoi_plot_2d
    # create voronoi diagram and get its finite vertices
    vor=Voronoi(points)
    print("vor",vor)
    fig = voronoi_plot_2d(vor,show_vertices=False, line_colors='black',line_width=2,line_alpha=1, point_size=0)
    import matplotlib.pyplot as plt
    plt.show()
    #print('vertices',vor.vertices)
    vx=np.zeros(len(vor.vertices))
    vy=np.zeros(len(vor.vertices))
    for i in range(0,len(vor.vertices)):
      vx[i]=vor.vertices[i,0]
      #vx=np.around(vx)
    #print("vx",vx)
    for i in range(0,len(vor.vertices)):
      vy[i]=vor.vertices[i,1]
      #vy=np.around(vy)
    #print("vy",vy)
    #Hier ist das Voronoi-Diagramm schon existent
# voronoi2mask.m:85
    # create a mask to draw the vertices
    #border=np.zeros((size,size))
# voronoi2mask.m:88
    #mask=np.zeros((size,size))
# voronoi2mask.m:89
    # draw vertices on mask

    #print('vyLineround=',vyLine)
    #print(vxLine.size)
    # voronoi2mask.m:105

    for s in range(0, vx.size, 1):
      vxLine,vyLine=makelinefun(vx,vy,size^2)
      vxLine=np.around(vxLine,0)
      vyLine=np.around(vyLine,0)
      #print('vxLinesize=',vxLine.size)
      if (vxLine[s]>0):
        #print("s",s)
        if (vxLine[s]< size):
          if (vyLine[s] > 0):
            if (vyLine[s] < size):
            #border1=int(vxLine[s])
            #border2=int(vyLine[s])
              newInd=np.ravel_multi_index([int(vxLine[s]),int(vyLine[s])],[size,size])
              border=np.reshape(border,(50176))
              newInd=int(newInd)
              border[newInd]=255
              border=border.reshape(224,224)
            #print('[vxLine[s],vyLine[s]=',[vxLine[s],vyLine[s]])
    return (border,fig)
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
#img=Image.fromarray(border)
#cv2_imshow(border)


In [ ]:
import random
from PIL import Image
from google.colab.patches import cv2_imshow
import os
from google.colab import drive
import glob
drive.mount('/content/drive/')
drive = '/content/drive/MyDrive/Adrian/Semantic Segmentation-Laura Ulrich/Voronoi'

def voronoi_finite_polygons_2d(vor, radius=None):
    """
    Reconstruct infinite voronoi regions in a 2D diagram to finite
    regions.
    Parameters
    ----------
    vor : Voronoi
        Input diagram
    radius : float, optional
        Distance to 'points at infinity'.
    Returns
    -------
    regions : list of tuples
        Indices of vertices in each revised Voronoi regions.
    vertices : list of tuples
        Coordinates for revised Voronoi vertices. Same as coordinates
        of input vertices, with 'points at infinity' appended to the
        end.
    """

    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")

    new_regions = []
    new_vertices = vor.vertices.tolist()

    center = vor.points.mean(axis=0)
    if radius is None:
        radius = vor.points.ptp().max()*2

    # Construct a map containing all ridges for a given point
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))

    # Reconstruct infinite regions
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]

        if all(v >= 0 for v in vertices):
            # finite region
            new_regions.append(vertices)
            continue

        # reconstruct a non-finite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]

        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                # finite ridge: already in the region
                continue

            # Compute the missing endpoint of an infinite ridge
            t = vor.points[p2] - vor.points[p1] # tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal

            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + direction * radius

            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())

        # sort region counterclockwise
        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:,1] - c[1], vs[:,0] - c[0])
        new_region = np.array(new_region)[np.argsort(angles)]

        # finish
        new_regions.append(new_region.tolist())

    return new_regions, np.asarray(new_vertices)
    # Generated with SMOP  0.41

import numpy as np
from numpy import random
import random
import pandas as pd

size=224
# test_voronoi_Mischbrueche.m:14
    # Schleife zum Erzeugen 20 synthetischer Mischbruchbilder
points=[[0,0]]
xpoints=[[0]]
ypoints=[[0]]
for u in range(1,20):
    # Zufallszahlen für Voronoi erzeugen; random number zwischen 5 und 10
    # für 5-10 seed points bzw. Körner
    a=5
# test_voronoi_Mischbrueche.m:21
    b=10

    c=3
    d_duk=len(duktil_path)
    d_spr=len(spröde_path)
# test_voronoi_Mischbrueche.m:22
    r=round((b - a)*random.random() + a)

# test_voronoi_Mischbrueche.m:23
    x1=random.random()*size
    xpoints.append([x1])
    #print("xpoints",xpoints)
# test_voronoi_Mischbrueche.m:24
    y1=random.random()*size
    ypoints.append([y1])
    points.append([x1,y1])
#print("points=",points)
# test_voronoi_Mischbrueche.m:25
    # Voronoi Maske erstellen
from scipy.spatial import Voronoi, voronoi_plot_2d
xpoints=np.delete(xpoints,0,axis=0)
#print(xpoints)
ypoints=np.delete(ypoints,0,axis=0)
#print("xpoints",xpoints)
#print("points",points)
#print("points",points)
vor=Voronoi(points)
#print("vor",vor)
regions, vertices = voronoi_finite_polygons_2d(vor)
#print "--"
#print ("region",regions)
#print ("regions[2]",regions[2])
#print ("regions",regions)
#print "--"
#print vertices
# colorize
random.shuffle(regions)
#print("regions",regions)

from PIL import Image, ImageDraw, ImageFilter, ImageOps
import random
from PIL import Image
import os
from google.colab import drive
from google.colab.patches import cv2_imshow
from google.colab import files
import glob

drive = '/content/drive/MyDrive/Adrian/Semantic Segmentation-Laura Ulrich/Voronoi'
Maskenbausteine1_bw_path = os.path.join(drive, 'Maskenbausteine','Klasse1','Masken_BW')
Maskenbausteine2_bw_path = os.path.join(drive, 'Maskenbausteine','Klasse2','Masken_BW')

back_im = Image.new("1", (224,224), 0)
for j in range (4000,6000):
  for i in range(len(regions)):
    if random.uniform(0, 1)>=0.5:
      drive = '/content/drive/MyDrive/Adrian/Semantic Segmentation-Laura Ulrich/Voronoi'
    #Speicherort
      Maskenbausteine1_path=os.path.join(drive, 'Maskenbausteine','Klasse1')
    #Maskenbausteine2_path=os.path.join(drive, 'Maskenbausteine','Klasse2')
    #Datenursprung
      duk_path="/content/drive/MyDrive/Adrian/MLSeminar/TP1:Trainingdata/1.3.2.1 EB duktil 224x224/"
      duk_file = random.choice(os.listdir("/content/drive/MyDrive/Adrian/MLSeminar/TP1:Trainingdata/1.3.2.1 EB duktil 224x224/"))
    #mask = Image.fromarray(mask)
      polygon=vertices[regions[i]]
      plt.fill(*zip(*polygon), 'red')
      plt.xlim(vor.min_bound[0] - 0.1, vor.max_bound[0] + 0.1)
      plt.ylim(vor.min_bound[1] - 0.1, vor.max_bound[1] + 0.1)
      plt.axis('off')
      #"Klasse 1 binary, Maskendurchlauf fertig"
      polygon=vertices[regions[i]]
      plt.fill(*zip(*polygon), 'black')
      plt.xlim(vor.min_bound[0] - 0.1, vor.max_bound[0] + 0.1)
      plt.ylim(vor.min_bound[1] - 0.1, vor.max_bound[1] + 0.1)
      fig=plt.axis('off')
      plt.savefig(os.path.join(Maskenbausteine1_path,"mask_reshape2.jpg"), quality=100, subsampling=0, pad_inches = 0, bbox_inches='tight')
      plt.show()

    ################## Überlagerndes Bild laden
      I = Image.open(os.path.join(duk_path,duk_file))
    #print("Hier ist I")
    #display(I)
      plt.axis('off')
      #back_reshape = Image.open(os.path.join(Maskenbausteine1_bw_path,"Result100.jpg")) #weg?
    #back_reshape = back_reshape.resize((224,224)) weg?
    #print("Hier ist back_reshape")
    #display(back_reshape)


      mask = Image.open(os.path.join(Maskenbausteine1_path,"mask_reshape2.jpg"))
      mask = mask.resize((224,224))
      mask = mask.convert('L')
      #print('Hier ist mask')
      mask_invert=ImageOps.invert(mask)
      #display(mask_invert)
      back_reshape.paste(I, (0,0), mask_invert)
    #print('Hier ist Ergebnis+duk')
      back_reshape.save(os.path.join(Maskenbausteine1_bw_path,"Result"+str(j)+".jpg"), dpi=(2000, 2000),quality='web_maximum', subsampling=0)
      back_reshape.save(os.path.join(Maskenbausteine1_bw_path,"Result"+str(j)+".png"), dpi=(2000, 2000),quality='web_maximum', subsampling=0)
    #display(back_reshape)

    #####Masken schwarz-weiß
      #back_bw = Image.open(os.path.join(Maskenbausteine1_bw_path,"Result_bw100.jpg"))
      #back_bw = back_bw.resize((224,224))
      Image_black = Image.open(os.path.join(Maskenbausteine1_bw_path,"I_black.jpg"))
      Image_black = Image_black.resize((224,224))
      back_bw.paste(Image_black, (0,0), mask_invert)
      back_bw.save(os.path.join(Maskenbausteine1_bw_path,"Result_bw"+str(j)+".jpg"), dpi=(2000, 2000),quality='web_maximum', subsampling=0)
      back_bw.save(os.path.join(Maskenbausteine1_bw_path,"Result_bw"+str(j)+".png"), dpi=(2000, 2000),quality='web_maximum', subsampling=0)
    #print('Result_bw+duk')
    #display(back_bw)


    else:
      drive = '/content/drive/MyDrive/Adrian/Semantic Segmentation-Laura Ulrich/Voronoi'
      Maskenbausteine1_path=os.path.join(drive, 'Maskenbausteine','Klasse1')
      Maskenbausteine2_path=os.path.join(drive, 'Maskenbausteine','Klasse2')
      spr_path='/content/drive/MyDrive/Adrian/MLSeminar/TP1:Trainingdata/1.3.2.2 EB spröde 224x224/'
      spr_file = random.choice(os.listdir('/content/drive/MyDrive/Adrian/MLSeminar/TP1:Trainingdata/1.3.2.2 EB spröde 224x224/'))
    #mask = Image.fromarray(mask)
      polygon=vertices[regions[i]]
      plt.fill(*zip(*polygon), 'red')
      plt.xlim(vor.min_bound[0] - 0.1, vor.max_bound[0] + 0.1)
      plt.ylim(vor.min_bound[1] - 0.1, vor.max_bound[1] + 0.1)
      plt.axis('off')
      #"Klasse 2 binary, Maskendurchlauf fertig"
      polygon=vertices[regions[i]]
      plt.fill(*zip(*polygon), 'black')
      plt.xlim(vor.min_bound[0] - 0.1, vor.max_bound[0] + 0.1)
      plt.ylim(vor.min_bound[1] - 0.1, vor.max_bound[1] + 0.1)
      fig=plt.axis('off')
      plt.savefig(os.path.join(Maskenbausteine2_path,"mask_reshape2.jpg"), quality=100, subsampling=0, pad_inches = 0,  bbox_inches='tight')
      plt.show()

    ################## Überlagerndes Bild laden
      I = Image.open(os.path.join(spr_path,spr_file))
      I = I.resize((224,224))
    #print("Hier ist I")
    #display(I)
      plt.axis('off')
      #back_reshape = Image.open(os.path.join(Maskenbausteine1_bw_path,"Result100.jpg")) #weg?
    #back_reshape = back_reshape.resize((224,224)) weg?
    #print("Hier ist back_reshape")
    #display(back_reshape)


      mask = Image.open(os.path.join(Maskenbausteine2_path,"mask_reshape2.jpg"))
      mask = mask.resize((224,224))
      mask = mask.convert('L')
    #print('Hier ist mask')
      mask_invert=ImageOps.invert(mask)
    #display(mask_invert)

      back_reshape.paste(I, (0,0), mask_invert)
    #print('Hier ist Ergebnis +spr')
      back_reshape.save(os.path.join(Maskenbausteine1_bw_path,"Result"+str(j)+".jpg"), dpi=(2000, 2000),quality='web_maximum', subsampling=0)
      back_reshape.save(os.path.join(Maskenbausteine1_bw_path,"Result"+str(j)+".png"), dpi=(2000, 2000),quality='web_maximum', subsampling=0)
    #display(back_reshape)

    #####Masken schwarz-weiß
      #back_bw = Image.open(os.path.join(Maskenbausteine1_bw_path,"Result_bw100.jpg"))
      #back_bw = back_bw.resize((224,224))
      Image_white = Image.open(os.path.join(Maskenbausteine1_bw_path,"I_white.jpg"))
      Image_white = Image_white.resize((224,224))
      back_bw.paste(Image_white, (0,0), mask_invert)
      back_bw.save(os.path.join(Maskenbausteine1_bw_path,"Result_bw"+str(j)+".jpg"), dpi=(2000, 2000),quality='web_maximum', subsampling=0)
      back_bw.save(os.path.join(Maskenbausteine1_bw_path,"Result_bw"+str(j)+".png"), dpi=(2000, 2000),quality='web_maximum', subsampling=0)
  print("Hier ist Ergebnis")
  display(back_reshape)
  print('Result_bw+spr')
  display(back_bw)